# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [12]:
import warnings
warnings.filterwarnings('ignore')

In [13]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [14]:
#!pip install pandas

In [20]:
import pandas as pd
df = pd.read_csv('data/Data.csv')

In [21]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [ ]:
#!pip install langchain_community

In [25]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [26]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=None)


In [27]:
prompt = ChatPromptTemplate.from_template( "Write persuasive product description for the following item:\nProduct: {product_name}"
)

In [28]:

chain = LLMChain(llm=llm, prompt=prompt)

C:\Users\Lsula\AppData\Local\Temp\ipykernel_53144\546483037.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [30]:
product = "drawingnote"
response =chain.run(product)

In [31]:
print("\nProduct Description Response:\n", response)


Product Description Response:
 Are you tired of carrying around bulky sketch pads and messy notebooks? Say goodbye to clutter and hello to convenience with our innovative Drawingnote! This sleek and compact drawing notebook is perfect for artists on the go. It features a smooth, blank paper surface that is perfect for sketching, doodling, journaling, and more. The durable cover and high-quality paper ensure that your artwork will stay protected and pristine. Plus, the compact size makes it easy to slip into your bag or backpack for creative inspiration wherever you go. Upgrade your artistic practice with the Drawingnote today!


## SimpleSequentialChain

In [32]:
from langchain.chains import SimpleSequentialChain

In [34]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "You are a shopping assistant. Write a product description for the following item:\nProduct: {product_name}")
    
    

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [35]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "based on the product description, convense the customer to buy for the following item:\nProduct: {product_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [36]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [37]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Introducing the DrawingNote, the perfect companion for all your artistic endeavors! This sleek and stylish notebook features high-quality blank pages for you to sketch, doodle, and draw to your heart's content. Whether you're a seasoned artist or just starting out, the DrawingNote is the ideal canvas for expressing your creativity. Its compact size makes it easy to take on the go, so you can capture your ideas anytime, anywhere. Elevate your art game with the DrawingNote today!
The DrawingNote is a must-have for any creative individual looking to express their artistic talent on the go. With its high-quality blank pages and compact size, this sleek notebook is the perfect canvas for all your drawing and doodling needs. Whether you're a seasoned artist or a beginner, the DrawingNote will help you elevate your art game and capture your ideas anytime, anywhere. Don't miss out on this perfect companion for your artistic endeavors - get your h

"The DrawingNote is a must-have for any creative individual looking to express their artistic talent on the go. With its high-quality blank pages and compact size, this sleek notebook is the perfect canvas for all your drawing and doodling needs. Whether you're a seasoned artist or a beginner, the DrawingNote will help you elevate your art game and capture your ideas anytime, anywhere. Don't miss out on this perfect companion for your artistic endeavors - get your hands on the DrawingNote today and unleash your creativity!"

**Repeat the above twice for different products**

## SequentialChain

In [38]:
from langchain.chains import SequentialChain

In [39]:
llm = ChatOpenAI(temperature=0.9)


first_prompt = ChatPromptTemplate.from_template(
"Translate the following product review into Arabic:\n\nReview: {review}"
)

chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="Arabic_Review"
                    )


In [40]:
second_prompt = ChatPromptTemplate.from_template(
    "Suggest some similar product name for the  {Arabic_Review}"
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="similar_products"
                    )


In [41]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
    "Summarize the following product review in one line:  {similar_products}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="summary"
                      )


In [42]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
        "Translate the following product review into English:\n\nReview: {summary}"
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="English_Review"
                     )


In [43]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["review"],
    output_variables=["Arabic_Review", "similar_products", "summary", "English_Review"],
    verbose=True
)

In [44]:
review = df.Review[5]
overall_chain(review)

C:\Users\Lsula\AppData\Local\Temp\ipykernel_53144\1992003631.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  overall_chain(review)




> Entering new SequentialChain chain...

> Finished chain.


{'review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'Arabic_Review': 'مراجعة: أجد أن الطعم سيء. لا يدوم الرغوة، هذا غريب. أشتري نفس الأشياء من المتاجر والطعم أفضل بكثير...\nهل هي دفعة قديمة أو تقليد؟',
 'similar_products': '1.  مراجعة: تجربة سيئة. الرغوة غير دائمة والطعم غير مقبول.\n2.  مراجعة: تجربة مخيبة. النكهة غير مقنعة والرغوة ضعيفة.\n3.  مراجعة: تجربة مخيبة للأمل. الطعم سيء والرغوة تختفي بسرعة.\n4.  مراجعة: لا يستحق الشراء. الطعم لا يُرضي الذوق والجودة تتراجع.\n5.  مراجعة: تجربة سلبية. الطعم سيء والرغوة غير مرضية.',
 'summary': 'Product review summary: Disappointing experience with bad taste and weak foam.',
 'English_Review': 'Translation: Review: Overall disappointing experience with unpleasant taste and weak foam.'}

**Repeat the above twice for different products or reviews**

## Router Chain

In [45]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

biology_template = """You are an excellent biologist. \
You have a deep understanding of living organisms, \
from the molecular and cellular level to entire ecosystems. \
You are skilled at observing patterns in nature, analyzing biological data, \
and explaining complex processes like evolution, genetics, physiology, and ecology. \
You can clearly communicate how life functions and adapts, \
and you make connections between different biological concepts \
to answer challenging questions.

Here is a question:
{input}"""

In [46]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    },
    {
        "name": "biology",
        "description": "Good for answering biology questions",
        "prompt_template": biology_template
    }
]

In [47]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [48]:
llm = ChatOpenAI(temperature=0)

In [49]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [50]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [51]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [52]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [53]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

C:\Users\Lsula\AppData\Local\Temp\ipykernel_53144\3038952769.py:1: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


In [54]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation is the electromagnetic radiation emitted by a perfect absorber and emitter of radiation, known as a black body. A black body absorbs all radiation that falls on it and emits radiation across the entire electromagnetic spectrum. The spectrum of black body radiation is continuous and depends only on the temperature of the black body. This phenomenon is described by Planck's law, which states that the intensity of radiation emitted by a black body at a given wavelength is proportional to the temperature of the body and the wavelength raised to the fifth power."

In [55]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'The answer to 2 + 2 is 4.'

In [56]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


'Every cell in our body contains DNA because DNA is the genetic material that carries the instructions for the development, functioning, and reproduction of all living organisms. DNA contains the information needed to build and maintain an organism, including the proteins that make up our cells and tissues. \n\nHaving DNA in every cell ensures that each cell has the necessary genetic information to carry out its specific functions and to replicate itself accurately during cell division. This ensures that the genetic information is passed on to the next generation of cells. \n\nAdditionally, DNA is constantly being used by cells to carry out processes such as protein synthesis, cell division, and repair. Having DNA in every cell allows for the coordination of these processes and ensures that the organism functions properly as a whole. \n\nIn summary, every cell in our body contains DNA because it is essential for the proper functioning and development of all living organisms.'

**Repeat the above at least once for different inputs and chains executions - Be creative!**

In [57]:
chain.run("What is bermuda triangle, and what would happen to if it approached one?")



> Entering new MultiPromptChain chain...
None: {'input': 'What is bermuda triangle, and what would happen to if it approached one?'}
> Finished chain.


"The Bermuda Triangle, also known as the Devil's Triangle, is a loosely defined region in the western part of the North Atlantic Ocean where a number of aircraft and ships are said to have disappeared under mysterious circumstances. The boundaries of the Bermuda Triangle vary depending on the source, but it is generally considered to be bounded by Miami, Bermuda, and Puerto Rico.\n\nIf a person or object approached the Bermuda Triangle, there is no scientific evidence to suggest that anything unusual or mysterious would happen. The disappearances and incidents that have been attributed to the Bermuda Triangle are often the result of natural phenomena such as severe weather, human error, or mechanical failures.\n\nIn reality, the Bermuda Triangle is no more dangerous than any other part of the ocean, and millions of ships and aircraft safely navigate through the area every year without incident."